In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import cross_validate
import re
import plotly.graph_objects as go
import plotly.express as px
from sklearn.preprocessing import OneHotEncoder
import re
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [2]:
data = pd.read_csv('processed_df.csv')

**ВЫКИНЕМ НАНЫ**

In [3]:
data_no_nans = data.copy()
data_no_nans = data_no_nans.dropna()
data_no_nans.isna().sum()

Unnamed: 0              0
Maruti                  0
Skoda                   0
Honda                   0
Hyundai                 0
Toyota                  0
Ford                    0
Renault                 0
Mahindra                0
Tata                    0
Chevrolet               0
Fiat                    0
Datsun                  0
Jeep                    0
Mercedes-Benz           0
Mitsubishi              0
Audi                    0
Volkswagen              0
BMW                     0
Nissan                  0
Lexus                   0
Jaguar                  0
Land                    0
MG                      0
Volvo                   0
Daewoo                  0
Kia                     0
Force                   0
Ambassador              0
Ashok                   0
Isuzu                   0
Opel                    0
year                    0
selling_price           0
km_driven               0
Diesel                  0
Individual              0
Dealer                  0
Manual      

**ВЫДЕЛИМ ТРЕНИРОВОЧНЫЕ И ТЕСТОВЫЕ МАССИВЫ**

In [4]:
train_data_no_nans = data_no_nans.sample(frac=0.8)
test_data_no_nans = data_no_nans.iloc[~data_no_nans.index.isin(train_data_no_nans.index), :]

In [5]:
features_train = train_data_no_nans.iloc[:, train_data_no_nans.columns != 'selling_price']
responses_train = train_data_no_nans['selling_price'].to_frame(name='selling_price')
features_test = test_data_no_nans.iloc[:, test_data_no_nans.columns != 'selling_price']
responses_test = test_data_no_nans['selling_price'].to_frame(name='selling_price')

**НОРМИРУЕМ ДАННЫЕ**

In [6]:
scaller = StandardScaler()
x_train = scaller.fit_transform(features_train)
y_train = scaller.fit_transform(responses_train)
x_test = scaller.fit_transform(features_test)
y_test = scaller.fit_transform(responses_test)

# **ОБУЧИМ МОДЕЛЬ НА ТЕХ ДАННЫХ, ЧТО ЕСТЬ**

**ОБУЧИМ МОДЕЛЬ С ЛИНЕАРИЗАЦИЕЙ КОЭФФИЦИЕНТОВ L1 LASSO**

In [7]:
l1_regr = Lasso(1e-5, max_iter=10 ** 4)
l1_regr.fit(x_train, y_train)
l1_regr.score(x_train, y_train)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.366e+01, tolerance: 6.277e-01
  model = cd_fast.enet_coordinate_descent(


0.859491299544503

In [8]:
l1_regr.coef_

array([-2.03505348e-03,  4.11560248e-02, -8.58642939e-03,  5.17813196e-03,
       -1.45990575e-03,  8.89397134e-02, -2.83437764e-03, -4.06491985e-03,
       -4.84615248e-03, -3.84673216e-02, -1.71904215e-02, -7.44623233e-03,
       -1.28646587e-02,  6.07820098e-02,  9.80734320e-02,  1.41460783e-02,
        8.39099127e-02, -2.32790650e-02,  4.16201892e-01, -1.21584889e-03,
        3.05805619e-01,  1.60285742e-01,  7.69663291e-02,  1.11151402e-02,
        1.01619022e-01,  1.01755932e-02,  1.24045952e-02, -1.73864362e-03,
        1.29370137e-02,  4.74143978e-04,  1.77668300e-02,  5.67717534e-03,
        2.08273173e-01, -3.64132587e-02, -4.73133164e-02, -1.90378046e-03,
        3.69938350e-02, -4.69406348e-02, -1.44356577e+00, -1.35771515e+00,
       -7.58471341e-01, -4.43313116e-01, -7.17546980e-03,  4.51193599e-02,
        1.93592412e-01,  1.06683443e-01, -5.27256296e-02, -1.03762118e-02])

In [9]:
l1_regr.score(x_test, y_test)

0.7711877864126424

**КАК ВИДНО НА ТЕСТОВОЙ ВЫБОРКЕ РЕЗУЛЬТАТ ДАЖЕ ЛУЧШЕ.**

**МФ ПОЛУЧИЛИ РЕЗУЛЬТАТ ПО МЕТРИКЕ R2**

**ПОПРОБУЕМ ОЦЕНИТЬ С ПОМОЩЬЮ МЕТРИКИ MAE**

In [10]:
y_train_predict = l1_regr.predict(x_train)
y_test_predict = l1_regr.predict(x_test)
loss_train = mean_absolute_error(y_train, y_train_predict)
loss_test = mean_absolute_error(y_test, y_test_predict)

In [11]:
print(loss_train)
print(loss_test)

0.19404398368654388
0.22757332857314572


**ПОСМОТРИМ СКОЛЬКО ЭТО В ДЕНЕЖНОМ ЭКВИВАЛЕНТЕ С ПОМОЩЬЮ ИНВЕРСИИ НОРМАЛИЗАЦИИ**

In [12]:
print(scaller.inverse_transform([[loss_train]]))
print(scaller.inverse_transform([[loss_test]]))

[[729172.83906247]]
[[752276.46295742]]


**ОЦЕНИМ С ПОМОЩЬЮ МЕТРИКИ MAPE**

In [13]:
loss_train_mape = mean_absolute_percentage_error(y_train, y_train_predict)
loss_test_mape = mean_absolute_percentage_error(y_test, y_test_predict)

In [14]:
print(loss_train_mape)
print(loss_test_mape)

2.2920453766236712
1.6110357103332713


**ОПЯТЬ ЗАМЕЧАЕМ, ЧТО НА ТЕСТОВЫХ ДАННЫХ РЕЗУЛЬТАТ ЛУЧШЕ**

# **ДОБАВИМ ПОЛИНОМ 9 СТЕПЕНИ ПО ПРИЗНАКАМ ТИП ТОПЛИВА И ОБЬЕМ ДВИГАТЕЛЯ**

**В РЕЗУЛЬТАТЕ АНАЛИЗА ДАННЫХ (ФАЙЛ processing_nes.ipynb) МЫ ВЫЯСНИЛИ, ЧТО ОБЬЕМ ДВИГАТЕЛЯ НАХОДИТСЯ В НЕКОТОРОЙ КОРЕЛЯЦИИ ОТ ТИПА ТОПЛИВА. ПОСТРОИВ ПОЛИНОМИАЛЬНУЮ МОДЕЛЬ, ОБУЧИВ ЕЕ И ПОСЧИТАВ ПОТЕРИ С КРОСС ВАЛИДАЦИЕЙ МЫ ПОНЯЛИ, ЧТО ДЛЯ ДВУХ ЭТИХ ДВУХ ПРИЗНАКОВ С ПОЛИНОМОМ СТЕПЕНЬЮ 9 (ФАЙЛ analyse.ipynb) МЫ МОЖЕМ УЛУЧШИТЬ МОДЕЛЬ. ПОПРОБУЕМ ЕЕ УЛУЧШИТЬ ДОБАВИВ ЭТИ ПРИЗНАКИ**

**ВОЗЬМЕМ ФРЕЙМ БЕЗ НАНОВ И СКАЛЯРИЗУЕМ**

In [15]:
scaller_poly = StandardScaler()
x_poly = scaller_poly.fit_transform(data_no_nans[data_no_nans.columns.drop('selling_price')])
y_poly = scaller_poly.fit_transform(data_no_nans[['selling_price']])

In [16]:
scaled_df = pd.DataFrame(data=np.hstack((y_poly, x_poly)), columns=['selling_price'] + list(data_no_nans.columns.drop('selling_price')))
scaled_df

,selling_price,Unnamed: 0,Maruti,Skoda,Honda,Hyundai,Toyota,Ford,Renault,Mahindra,...,First Owner,Second Owner,Third Owner,Fourth & Above Owner,milage,engine,max_power,torque,RPM_min,seats
0,-0.237694,-1.734017,1.535349,-0.115902,-0.251284,-0.454241,-0.253854,-0.228089,-0.173,-0.327019,...,0.721895,-0.587260,-0.263667,-0.144741,1.021092,-0.418993,-0.495635,0.151889,-0.617447,-0.436655
1,-0.339271,-1.733585,-0.651318,8.627996,-0.251284,-0.454241,-0.253854,-0.228089,-0.173,-0.327019,...,-1.385243,1.702822,-0.263667,-0.144741,0.440392,0.077846,0.345081,0.827818,-1.065356,-0.436655
2,-0.608449,-1.733153,-0.651318,-0.115902,3.979562,-0.454241,-0.253854,-0.228089,-0.173,-0.327019,...,-1.385243,-0.587260,3.792666,-0.144741,-0.443505,0.075858,-0.381717,-0.586450,0.009625,-0.436655
3,-0.523379,-1.732722,-0.651318,-0.115902,-0.251284,2.201476,-0.253854,-0.228089,-0.173,-0.327019,...,0.721895,-0.587260,-0.263667,-0.144741,0.918313,-0.124865,-0.039962,0.484446,-0.841402,-0.436655
4,-0.644001,-1.732290,1.535349,-0.115902,-0.251284,-0.454241,-0.253854,-0.228089,-0.173,-0.327019,...,0.721895,-0.587260,-0.263667,-0.144741,-0.854619,-0.319626,-0.091225,-0.718932,1.622096,-0.436655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7841,-0.402756,1.731147,-0.651318,-0.115902,-0.251284,2.201476,-0.253854,-0.228089,-0.173,-0.327019,...,0.721895,-0.587260,-0.263667,-0.144741,-0.237947,-0.520348,-0.243591,-0.707667,1.174187,-0.436655
7842,-0.637652,1.731579,-0.651318,-0.115902,-0.251284,2.201476,-0.253854,-0.228089,-0.173,-0.327019,...,-1.385243,-0.587260,-0.263667,6.908901,-0.674757,0.067909,0.529629,0.661089,-0.707029,-0.436655
7843,-0.324034,1.732011,1.535349,-0.115902,-0.251284,-0.454241,-0.253854,-0.228089,-0.173,-0.327019,...,0.721895,-0.587260,-0.263667,-0.144741,-0.032390,-0.418993,-0.498483,0.151889,-0.617447,-0.436655
7844,-0.440848,1.732442,-0.651318,-0.115902,-0.251284,-0.454241,-0.253854,-0.228089,-0.173,-0.327019,...,0.721895,-0.587260,-0.263667,-0.144741,1.064773,-0.124865,-0.609553,-0.411385,-0.796611,-0.436655


**СОЗДАДИМ ПОЛИНОМ 9Й СТЕПЕНИ ДЛЯ ДВИГАТЕЛЯ И ДИЗЕЛЯ**

In [17]:
poly_obj = PolynomialFeatures(7)
engine_diesel_poly = poly_obj.fit_transform(scaled_df[['Diesel', 'engine']])
engine_diesel_poly.shape

(7846, 36)

**УБЕРЕМ ПЕРВЫЕ ТРИ СТОЛБЦА, ПЕРВЫЙ ПОТОМУ ЧТО ЕДИНИЧНЫЙ ДЛЯ СВОБОНОГО ВЕСА, ВТОРЫЕ ДВА ПОТОМУ ЧТО ПОВТОРЕНИЕ ТЕХ, ЧТО УЖЕ ЕСТЬ В ДАТАСЕТЕ, ЗАПИШЕМ ВО ФРЕЙМ**

In [18]:
engine_diesel_poly_df = pd.DataFrame(data=engine_diesel_poly[:, 3:], columns=[f'Dies*Engine{i + 1}' for i in range(engine_diesel_poly.shape[1] - 3)])

**СОБЕРЕМ ПОЛИНОМ С ДАТАСЕТОМ**

In [19]:
poly_prepared = pd.concat([scaled_df, engine_diesel_poly_df], axis=1)

**РАЗДЕЛИМ ВЕСЬ СКАЛЯРИЗОВАННЫЙ ДАТАСЕТ НА ТРЕНИРОВОЧНЫЙ И ТЕСТОВЫЙ**

In [20]:
poly_prepared_train = poly_prepared.sample(frac=0.8, random_state=0).sort_index()
poly_prepared_test = poly_prepared.iloc[~scaled_df.index.isin(poly_prepared_train.index)].sort_index()

**НАЧНЕМ ОБУЧАТЬ МОДЕЛЬ С L1/L2 НА ТРЕНИРОВОЧНОМ ДАТАСЕТЕ**

In [21]:
regr_poly = Ridge(alpha=0.0001, max_iter=10 ** 4)
regr_poly.fit(poly_prepared_train.iloc[:, 1:], poly_prepared_train.iloc[:, 0])

Ridge(alpha=0.0001, max_iter=10000)

In [22]:
regr_poly.coef_

array([ 9.83813174e-04, -3.15763859e-02, -1.86235097e-02, -2.38158744e-02,
       -6.12217933e-02,  5.22286535e-02, -3.47338973e-02, -3.86751207e-02,
       -4.78899322e-02, -7.57828284e-02, -4.27670335e-02, -2.08825217e-02,
       -2.91704802e-02,  2.22248922e-02,  1.09301135e-01,  5.95599857e-03,
        7.40423838e-02, -4.58662268e-02,  3.77219617e-01, -2.13482766e-02,
        2.71358006e-01,  1.49099507e-01,  6.96460680e-02,  7.08189898e-03,
        9.72699479e-02,  6.07472124e-03,  1.28128967e-02, -5.36287258e-03,
        1.46477791e-02, -1.59307343e-26,  1.93191433e-02,  4.73656601e-03,
        2.15386081e-01, -6.49711417e-02, -2.85290841e-03,  4.91701141e-03,
        4.13029703e-02, -2.89593151e-02, -1.51062182e+00, -1.42497831e+00,
       -7.92538956e-01, -4.53416679e-01, -3.93317800e-02, -2.22783798e-02,
        1.85902963e-01,  2.07792988e-01, -2.42722655e-02, -2.56354273e-03,
        5.68590085e-04, -5.94033864e-03, -7.71605054e-02, -2.96623244e-03,
       -2.10946017e-02, -

**ОЦЕНИМ ПОТЕРИ ПО МЕТРИКЕ MAE**

In [23]:
y_train_predict = regr_poly.predict(poly_prepared_train.iloc[:, 1:])
y_test_predict = regr_poly.predict(poly_prepared_test.iloc[:, 1:])
loss_train = mean_absolute_error(poly_prepared_train.iloc[:, 0], y_train_predict)
loss_test = mean_absolute_error(poly_prepared_test.iloc[:, 0], y_test_predict)

In [24]:
print(loss_train)
print(loss_test)

0.19380667513231625
0.196529693134915


In [25]:
print(scaller_poly.inverse_transform([[loss_train]]))
print(scaller_poly.inverse_transform([[loss_test]]))

[[789842.40029945]]
[[791987.00090534]]


**ОЦЕНИМ ПОТЕРИ ПО МЕТРИКЕ MAPE**

In [26]:
loss_train_mape = mean_absolute_percentage_error(poly_prepared_train.iloc[:, 0], y_train_predict)
loss_test_mape = mean_absolute_percentage_error(poly_prepared_test.iloc[:, 0], y_test_predict)
print(loss_train_mape)
print(loss_test_mape)

1.169829741804977
1.3212712425316433


**КАК ВИДНО РЕЗУЛЬТАТЫ ЧУУУУУУУТЬ ЧУТЬ УЛУЧШИЛИСЬ, ПОПРОБУЕМ ЕЩЕ УЛУЧШИТЬ МОДЕЛЬ**